# Import Libraries

In [37]:
#province = 'krabi'
#province = 'bangkok'
province = 'nakhon'
#setting = 'DFMet'
setting = 'DFMetCD'
#setting = 'DFCD'

In [38]:
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import numpy as np
import pandas as pd

#from google.colab import drive
#drive.mount('/content/gdrive')

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
  

# Data Preparation

df_train = pd.read_csv('./Data/train_'+province+'_dist_combined_mavg4.csv')
df_test = pd.read_csv('./Data/test_'+province+'_dist_combined_mavg4.csv')
attr_train = df_train.iloc[:,np.r_[18, 19]]
attr_train.head()

,DFma_wm5,DFma_wm6
0,0.130605,0.111947
1,0.130605,0.130605
2,0.111947,0.130605
3,0.093289,0.111947
4,0.093289,0.093289


In [39]:
# Data Preparation

df_train = pd.read_csv('./Data/train_krabi_dist_combined_mavg4.csv')
df_test = pd.read_csv('./Data/test_krabi_dist_combined_mavg4.csv')
head = df_test.iloc[:,1:4]

#for i in range (6):
i=5
################################ DF ONLY ###################################

attr_train = df_train.iloc[:,np.r_[18, 19]]
X_train = pd.concat([attr_train], axis=1)
y_train = df_train.iloc[:,12]

attr_test = df_test.iloc[:,np.r_[18, 19]]
X_test = pd.concat([attr_test], axis=1)
y_test = df_test.iloc[:,12]

rf = RandomForestRegressor(max_depth=30, random_state=0, n_estimators=10)
svr_lin = SVR(kernel='linear')
#svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[rf, rf], meta_regressor=svr_lin)

ensembles = stregr.fit(X_train, y_train)
y_hat = ensembles.predict(X_test)

y = pd.DataFrame(y_hat, columns = ['predicted'])
file = pd.concat([head, y_test,y], axis = 1)
file.columns = [['addrcode','Week','Year','actual','predicted']]
file.to_csv('./Results/District/'+setting+'/'+province+'_Dist_Ensemble_withoutST_DF%d.csv'%(i+1), encoding = 'utf-8')
print("DF%d without"%(i+1))

################################ WITH SETTING ###################################
# NST District JarPop9s, BinPop9s, Misc_tallPop9s 22, 26, 29
# bangkok District JarPop9s, Bucket, TirePop9s 26, 33, 28 
# Krabi District Jar, Bucket, Bin 33, 35, 31  

attr_train = df_train.iloc[:,np.r_[18, 19]]
# last_train = df_train.iloc[:,np.r_[20, 21]]
# last_train = df_train.iloc[:,np.r_[22, 26, 29]]
# last_train = df_train.iloc[:,np.r_[26, 33, 28]]
# last_train = df_train.iloc[:,np.r_[33, 35, 31]]
last_train = df_train.iloc[:,np.r_[20, 21, 22, 26, 29]]
# last_train = df_train.iloc[:,np.r_[20, 21, 26, 33, 28]]
# last_train = df_train.iloc[:,np.r_[20, 21, 33, 35, 31]]
X_train = pd.concat([attr_train,last_train], axis=1)
y_train = df_train.iloc[:,12]

attr_test = df_test.iloc[:,np.r_[18, 19]]
# last_test = df_test.iloc[:,np.r_[20, 21]]
# last_test = df_test.iloc[:,np.r_[22, 26, 29]]
# last_test = df_test.iloc[:,np.r_[26, 33, 28]]
# last_test = df_test.iloc[:,np.r_[33, 35, 31]]
last_test = df_test.iloc[:,np.r_[20, 21, 22, 26, 29]]
# last_test = df_test.iloc[:,np.r_[20, 21, 26, 33, 28]]
# last_test = df_test.iloc[:,np.r_[20, 21, 33, 35, 31]]
X_test = pd.concat([attr_test,last_test], axis=1)
y_test = df_test.iloc[:,12]

rf = RandomForestRegressor(max_depth=30, random_state=0, n_estimators=10)
svr_lin = SVR(kernel='linear')
svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[rf, rf], meta_regressor=svr_lin)

ensembles = stregr.fit(X_train, y_train)
y_hat = ensembles.predict(X_test)

y = pd.DataFrame(y_hat, columns = ['predicted'])
file = pd.concat([head, y_test,y], axis = 1)
file.columns = [['addrcode','Week','Year','actual','predicted']]
file.to_csv('./Results/District/'+setting+'/'+province+'_Dist_Ensemble_withST_DF%d.csv'%(i+1), encoding = 'utf-8')
print("DF%d with"%(i+1))

### **RF**

available_addr = pd.read_csv('./Results/District/'+setting+'/'+province+'_Dist_Ensemble_withST_DF6.csv')
a = available_addr['addrcode'].unique()

#for i in range (6):
i=5
addr = []
rmse_a = []
rmse_n = []
mae_a =[]
mae_n = []
smape_a = []
smape_n = []
im_mae=[]
im_rmse=[]
im_smape=[]

for j in range (len(a)):
    add = a[j]
    addr.append(add)

    #ADJUSTED CD
    adjustedCD = pd.read_csv('./Results/District/'+setting+'/'+province+'_Dist_Ensemble_withoutST_DF%d.csv'%(i+1))
    subset_ad =  adjustedCD.loc[adjustedCD['addrcode'] == add]
    predicted = subset_ad['predicted']
    actual = subset_ad['actual']

    # calculate RMSE
    rmse = np.sqrt(mean_squared_error(predicted, actual))
    rmse_a.append(rmse)

    # calculate MAE
    mae = mean_absolute_error(predicted,actual)
    mae_a.append(mae)

    Smape = smape(predicted, actual)
    smape_a.append(Smape)


    #NORMAL CD
    normalCD = pd.read_csv('./Results/District/'+setting+'/'+province+'_Dist_Ensemble_withST_DF%d.csv'%(i+1)) 
    subset_norm =  normalCD.loc[normalCD['addrcode'] == add]
    predicted_norm = subset_norm['predicted']
    actual_norm = subset_norm['actual']

    # calculate RMSE
    lin_mse = mean_squared_error(predicted_norm,actual_norm)
    rmse_norm = np.sqrt(lin_mse)
    rmse_n.append(rmse_norm)

    # calculate MAE
    mae_norm = mean_absolute_error(predicted_norm,actual_norm)
    mae_n.append(mae_norm)

    Smape_norm = smape(predicted_norm,actual_norm)
    smape_n.append(Smape_norm)

    #%improvement
    improve_smape = (Smape - Smape_norm)/Smape
    im_smape.append(improve_smape)

    improve_rmse = (rmse - rmse_norm)/rmse
    im_rmse.append(improve_rmse)

    improve_mae = (mae - mae_norm)/mae
    im_mae.append(improve_mae)


    code = pd.DataFrame({'addrcode':addr})

    smape_ad = pd.DataFrame({'smape_wo':smape_a})
    smape_norm = pd.DataFrame({'smape_w':smape_n})

    rmse_ad = pd.DataFrame({'rmse_wo': rmse_a})
    rmse_norm = pd.DataFrame({'rmse_w':rmse_n})

    mae_ad = pd.DataFrame({'mae_wo':mae_a})
    mae_norm = pd.DataFrame({'mae_w':mae_n})

    percent_smape = pd.DataFrame({'ims':im_smape}) 
    percent_rmse = pd.DataFrame({'imr':im_rmse}) 
    percent_mae = pd.DataFrame({'imm':im_mae}) 

    file = pd.concat([code, smape_ad, smape_norm,percent_smape, rmse_ad,rmse_norm,percent_rmse,mae_ad,mae_norm, percent_mae], axis=1)
    file.columns = [['addrcode','SMAPE without ST', 'SMAPE with ST','% improve SMAPE','RMSE without ST','RMSE with ST','% improvve RMSE','MAE without ST','MAE with ST','% improve MAE']]
    file.to_csv('./Results/District/'+setting+'/'+province+'_Dist_Ensemble_Eva_DF%d.csv'%(i+1), encoding = 'utf-8', index=False)

## **Sub district**

df_train = pd.read_csv('./Data/train_'+province+'_subdist_combined_mavg4.csv')
df_test = pd.read_csv('./Data/test_'+province+'_subdist_combined_mavg4.csv')
head = df_test.iloc[:,1:4]

#for i in range (6):
i=5

################################ DF ONLY ###################################
attr_train = df_train.iloc[:,18:19]
X_train = pd.concat([attr_train], axis=1)
y_train = df_train.iloc[:,12]

attr_test = df_test.iloc[:,18:19]
X_test = pd.concat([attr_test], axis=1)
y_test = df_test.iloc[:,12]

rf = RandomForestRegressor(max_depth=30, random_state=0, n_estimators=10)
svr_lin = SVR(kernel='linear')
#svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[rf, rf], meta_regressor=svr_lin)

ensembles = stregr.fit(X_train, y_train)
y_hat = ensembles.predict(X_test)

y = pd.DataFrame(y_hat, columns = ['predicted'])
file = pd.concat([head, y_test,y], axis = 1)
file.columns = [['addrcode','Week','Year','actual','predicted']]
file.to_csv('./Results/Subdistrict/'+setting+'/'+province+'_Subdist_Ensemble_withoutST_DF%d.csv'%(i+1), encoding = 'utf-8')
print("DF%d without"%(i+1))

################################ WITH Setting ###################################
# NST SubDistrict BinPop9s, BucketPop9s, Jar 22, 24, 35
# bangkok SubDistrict BowlPop9s, Misc_shorts, TirePop9s 23, 34, 28
# Krabi SubDistrict BinPop9s, TirePop9s, Misc_shortPop9s 22, 28, 25

attr_train = df_train.iloc[:,np.r_[18, 19]]
# last_train = df_train.iloc[:,np.r_[20, 21]]
# last_train = df_train.iloc[:,np.r_[22, 24, 35]]
# last_train = df_train.iloc[:,np.r_[23, 34, 28]]
# last_train = df_train.iloc[:,np.r_[22, 28, 25]]
last_train = df_train.iloc[:,np.r_[20, 21, 22, 24, 35]]
# last_train = df_train.iloc[:,np.r_[20, 21, 23, 34, 28]]
# last_train = df_train.iloc[:,np.r_[20, 21, 22, 28, 25]]
X_train = pd.concat([attr_train,last_train], axis=1)
y_train = df_train.iloc[:,12]

attr_test = df_test.iloc[:,np.r_[18, 19]]
# last_test = df_test.iloc[:,np.r_[20, 21]]
# last_test = df_test.iloc[:,np.r_[22, 24, 35]]
# last_test = df_test.iloc[:,np.r_[23, 34, 28]]
# last_test = df_test.iloc[:,np.r_[22, 28, 25]]
last_test = df_test.iloc[:,np.r_[20, 21, 22, 24, 35]]
# last_test = df_test.iloc[:,np.r_[20, 21, 23, 34, 28]]
# last_test = df_test.iloc[:,np.r_[20, 21, 22, 28, 25]]
X_test = pd.concat([attr_test,last_test], axis=1)
y_test = df_test.iloc[:,12]

rf = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=10)
svr_lin = SVR(kernel='linear')
svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[rf, rf], meta_regressor=svr_lin)

ensembles = stregr.fit(X_train, y_train)
y_hat = ensembles.predict(X_test)

y = pd.DataFrame(y_hat, columns = ['predicted'])
file = pd.concat([head, y_test,y], axis = 1)
file.columns = [['addrcode','Week','Year','actual','predicted']]
file.to_csv('./Results/Subdistrict/'+setting+'/'+province+'_Subdist_Ensemble_withST_DF%d.csv'%(i+1), encoding = 'utf-8')
print("DF%d with"%(i+1))


### **RF**

available_addr = pd.read_csv('./Results/Subdistrict/'+setting+'/'+province+'_Subdist_Ensemble_withST_DF6.csv')
a = available_addr['addrcode'].unique()

#for i in range (6):
i=5
addr = []
rmse_a = []
rmse_n = []
mae_a =[]
mae_n = []
smape_a = []
smape_n = []
im_mae=[]
im_rmse=[]
im_smape=[]

for j in range (len(a)):
    add = a[j]
    addr.append(add)

    #ADJUSTED CD
    adjustedCD = pd.read_csv('./Results/Subdistrict/'+setting+'/'+province+'_Subdist_Ensemble_withoutST_DF%d.csv'%(i+1))
    subset_ad =  adjustedCD.loc[adjustedCD['addrcode'] == add]
    predicted = subset_ad['predicted']
    actual = subset_ad['actual']

    # calculate RMSE
    rmse = np.sqrt(mean_squared_error(predicted, actual))
    rmse_a.append(rmse)

    # calculate MAE
    mae = mean_absolute_error(predicted,actual)
    mae_a.append(mae)

    Smape = smape(predicted, actual)
    smape_a.append(Smape)


    #NORMAL CD
    normalCD = pd.read_csv('./Results/Subdistrict/'+setting+'/'+province+'_Subdist_Ensemble_withST_DF%d.csv'%(i+1)) 
    subset_norm =  normalCD.loc[normalCD['addrcode'] == add]
    predicted_norm = subset_norm['predicted']
    actual_norm = subset_norm['actual']

    # calculate RMSE
    lin_mse = mean_squared_error(predicted_norm,actual_norm)
    rmse_norm = np.sqrt(lin_mse)
    rmse_n.append(rmse_norm)

    # calculate MAE
    mae_norm = mean_absolute_error(predicted_norm,actual_norm)
    mae_n.append(mae_norm)

    Smape_norm = smape(predicted_norm,actual_norm)
    smape_n.append(Smape_norm)

    #%improvement
    improve_smape = (Smape - Smape_norm)/Smape
    im_smape.append(improve_smape)

    improve_rmse = (rmse - rmse_norm)/rmse
    im_rmse.append(improve_rmse)

    improve_mae = (mae - mae_norm)/mae
    im_mae.append(improve_mae)


    code = pd.DataFrame({'addrcode':addr})

    smape_ad = pd.DataFrame({'smape_wo':smape_a})
    smape_norm = pd.DataFrame({'smape_w':smape_n})

    rmse_ad = pd.DataFrame({'rmse_wo': rmse_a})
    rmse_norm = pd.DataFrame({'rmse_w':rmse_n})

    mae_ad = pd.DataFrame({'mae_wo':mae_a})
    mae_norm = pd.DataFrame({'mae_w':mae_n})

    percent_smape = pd.DataFrame({'ims':im_smape}) 
    percent_rmse = pd.DataFrame({'imr':im_rmse}) 
    percent_mae = pd.DataFrame({'imm':im_mae}) 

    file = pd.concat([code, smape_ad, smape_norm,percent_smape, rmse_ad,rmse_norm,percent_rmse,mae_ad,mae_norm, percent_mae], axis=1)
    file.columns = [['addrcode','SMAPE without ST', 'SMAPE with ST','% improve SMAPE','RMSE without ST','RMSE with ST','% improvve RMSE','MAE without ST','MAE with ST','% improve MAE']]
    file.to_csv('./Results/Subdistrict/'+setting+'/'+province+'_Subdist_Ensemble_Eva_DF%d.csv'%(i+1), encoding = 'utf-8', index=False)


DF6 without
DF6 with
DF6 without
DF6 with
